In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import glob 
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os
import sys
import warnings
from keras.models import model_from_json

In [ ]:
#data, sampling_rate = librosa.load('/content/drive/My Drive/sample/WhatsApp Ptt 2020-10-10 at 5.28.20 PM.ogg')
ipd.Audio('/content/drive/My Drive/sample/WhatsApp Ptt 2020-10-10 at 5.30.49 PM.ogg')

In [ ]:
json_file = open('/content/drive/My Drive/AUG/model_json_aug.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/AUG/saved_models/Emotion_Model_aug.h5")

# the optimiser
opt = tf.keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:

X, sample_rate = librosa.load('/content/drive/My Drive/sample/WhatsApp Ptt 2020-10-10 at 5.30.49 PM.ogg'
                              ,res_type='kaiser_fast'
                              ,duration=2.5
                              ,sr=44100
                              ,offset=0.5
                             )

sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
newdf = pd.DataFrame(data=mfccs).T
newdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215
0,-6.70877,-7.422569,-8.63776,-6.099887,-5.436844,-5.943503,-8.254206,-10.242081,-9.885151,-6.906557,-5.983425,-3.029578,-1.873938,-3.424155,-5.288012,-6.475496,-6.307769,-4.631351,-4.085505,-4.768724,-3.072354,-0.31172,-0.284392,-3.146017,-5.689101,-5.36158,-4.828542,-5.32755,-1.358329,-0.230825,0.38034,0.614132,-0.291194,-3.813641,-2.976577,-1.987285,-2.082697,-1.131319,0.42377,-0.289635,...,-4.697441,-4.695516,-5.535732,-5.538737,-5.326729,-4.645002,-3.059988,-4.35269,-8.952375,-10.022192,-7.518728,-6.738299,-6.153591,-5.412624,-8.475688,-10.379973,-10.656225,-8.849454,-7.523123,-7.70182,-6.79228,-7.318581,-6.497276,-5.830551,-3.485508,-4.221387,-7.633576,-7.389871,-7.227342,-5.838514,-5.663262,-7.555474,-8.381383,-6.614356,-6.739068,-7.659476,-10.13311,-9.816707,-6.613499,-4.919301


In [ ]:
# Apply predictions
newdf= np.expand_dims(newdf, axis=2)
newpred = loaded_model.predict(newdf, 
                         batch_size=16, 
                         verbose=1)

newpred

1/1 [==============================] - 0s 2ms/step


array([[9.5739855e-20, 1.8849677e-08, 3.7334083e-15, 1.0438839e-25,
        5.0365813e-19, 8.3858604e-03, 5.5209177e-18, 1.5896252e-05,
        1.1031275e-05, 3.6551952e-01, 5.1730336e-07, 3.2550863e-05,
        6.2603456e-01, 2.7862034e-17]], dtype=float32)

In [ ]:
import pickle
filename = '/content/drive/My Drive/AUG/labels'
infile = open(filename,'rb')
lb = pickle.load(infile)
infile.close()

# Get the final predicted label
final = newpred.argmax(axis=1)
final = final.astype(int).flatten()
final = (lb.inverse_transform((final)))
print(final)

['male_sad']


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
